In [62]:
!nvidia-smi

Fri Dec  9 19:26:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:31:00.0 Off |                  Off |
| 30%   31C    P8    21W / 230W |   9817MiB / 24256MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:4B:00.0 Off |                  Off |
| 30%   

In [1]:
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import torch
print(torch.__version__)
torch.cuda.set_device(0)

1.13.0+cu117


In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.current_device()

In [ ]:
torch.version.cuda

In [3]:
import Models
#import in_gp_modified as in_gp
import in_gp_modified_2 as in_gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
import my_utils

In [4]:
ts_data = np.loadtxt(open("ts_data.csv","rb"), delimiter=",")

In [5]:
print(ts_data.shape)

(106251, 36)


# 0. Preprocess data that can be fed into models

### Define input and output window length

In [6]:
input_dim = 30
#output_dim = 4
output_dim = 1
T_span = input_dim + output_dim
n_id = 20

# 2. GP Prediction

## 2.2 INGP
#### parameterizes company_id & 30-day prices to predict future returns (<= 5d)
#### parameterize company id and day 1-30 prices, and then train_x are 2~35 time points, train_y are returns
#### in prediction, input can merely be a one-day time point, and output will be espective return

### 2.2.0 Data Preparation for INGP

In [7]:
# Differentiate X and y
z = ts_data[:,0:(input_dim+1)]
x = np.arange(T_span-1)
x = np.array([x for i in range(z.shape[0])])
y_price = ts_data[:,1:(T_span+1)]
y = np.diff(y_price, axis=1)/y_price[:,:-1]

# Train Test Split With All Companies Included
x_i = x[np.where(z[:,0]==0)]
y_i = y[np.where(z[:,0]==0)]
z_i = z[np.where(z[:,0]==0)]
np_z_train, np_z_test, np_x_train, np_x_test, np_y_train, np_y_test = train_test_split(z_i, x_i, y_i, test_size = 0.2, random_state = 42)
    
for i in range(1, n_id):
    x_i = x[np.where(z[:,0]==i)]
    y_i = y[np.where(z[:,0]==i)]
    z_i = z[np.where(z[:,0]==i)]
    np_z_train_i, np_z_test_i, np_x_train_i, np_x_test_i, np_y_train_i, np_y_test_i = train_test_split(z_i, x_i, y_i, test_size = 0.2, random_state = 42)
    
    np_z_train = np.concatenate((np_z_train, np_z_train_i), axis=0)
    np_z_test = np.concatenate((np_z_test, np_z_test_i), axis=0)
    np_x_train = np.concatenate((np_x_train, np_x_train_i), axis=0)
    np_x_test = np.concatenate((np_x_test, np_x_test_i), axis=0)
    np_y_train = np.concatenate((np_y_train, np_y_train_i), axis=0)
    np_y_test = np.concatenate((np_y_test, np_y_test_i), axis=0)

# Reshuffle Entire Datasets
idx = np.arange(np_z_train.shape[0])
np.random.shuffle(idx)
np_z_train, np_x_train, np_y_train = np_z_train[idx], np_x_train[idx], np_y_train[idx]

idx = np.arange(np_z_test.shape[0])
np.random.shuffle(idx)
np_z_test, np_x_test, np_y_test = np_z_test[idx], np_x_test[idx], np_y_test[idx]

In [8]:
# Standardize
np_z_train = (np_z_train - np.mean(z, axis=0)) / np.std(z, axis=0)
np_z_test = (np_z_test - np.mean(z, axis=0)) / np.std(z, axis=0)
np_y_train = (np_y_train - np.mean(y, axis=0)) / np.std(y, axis=0)
np_y_test = (np_y_test - np.mean(y, axis=0)) / np.std(y, axis=0)

In [9]:
# Convert to Torch Tensor
z_train = torch.tensor(np_z_train)
z_test = torch.tensor(np_z_test)
x_train = torch.tensor(np_x_train)
x_test = torch.tensor(np_x_test)
y_train = torch.tensor(np_y_train)
y_test = torch.tensor(np_y_test)

In [10]:
print(z_train.shape)
print(z_test.shape)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

torch.Size([84990, 31])
torch.Size([21261, 31])
torch.Size([84990, 30])
torch.Size([21261, 30])
torch.Size([84990, 30])
torch.Size([21261, 30])


In [39]:
print(np.unique(z[:,0]))
print(torch.unique(z_train[:,0]))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
tensor([-1.8107, -1.6341, -1.4575, -1.2809, -1.1043, -0.9277, -0.7511, -0.5746,
        -0.3980, -0.2214, -0.0448,  0.1318,  0.3084,  0.4850,  0.6616,  0.8381,
         1.0147,  1.1913,  1.3679,  1.5445])


### 2.2.1 Train INGP

In [11]:
id_mean = np.mean(z, axis=0)[0]
id_std = np.std(z, axis=0)[0]
print(id_mean)
print(id_std)
print(torch.unique(z_train[:,0])*id_std + id_mean)

10.253663494931812
5.662961347399698
tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19.], dtype=torch.float64)


In [14]:
INGP = in_gp.INGP(x_train, y_train, z_train, id_mean=id_mean, id_std=id_std, 
                  mean='zero', kernel='rbf', n_layers=1, hidden_dim=10)
loss = INGP.fit(x_train, y_train, z_train, niter=20, lr=0.001, batch_size=1000)

Iter 1/20 - Loss: 1411.757
Iter 2/20 - Loss: 1412.266
Iter 3/20 - Loss: 1433.714
Iter 4/20 - Loss: 1399.943
Iter 5/20 - Loss: 1454.145
Iter 6/20 - Loss: 1466.011
Iter 7/20 - Loss: 1369.028
Iter 8/20 - Loss: 1462.794
Iter 9/20 - Loss: 1411.471
Iter 10/20 - Loss: 1423.597
Iter 11/20 - Loss: 1441.161
Iter 12/20 - Loss: 1371.731
Iter 13/20 - Loss: 1398.458
Iter 14/20 - Loss: 1412.898
Iter 15/20 - Loss: 1454.919
Iter 16/20 - Loss: 1429.965
Iter 17/20 - Loss: 1417.336
Iter 18/20 - Loss: 1446.605
Iter 19/20 - Loss: 1438.039
Iter 20/20 - Loss: 1397.794


### 2.2.2 INGP Prediction

In [15]:
cond = input_dim-1
d = 1

In [16]:
# evaluate and test
# save trained params for MLP and set eval mode
y_train_pred_INGP_mean, y_train_pred_INGP_cov = INGP.predict(x_train, z_train, cond=cond, d=d)
y_test_pred_INGP_mean, y_test_pred_INGP_cov = INGP.predict(x_test, z_test, cond=cond, d=d)

In [ ]:
#print(y_train_pred_INGP[0:100])
#print(y_test_pred_INGP[0:100])
#print(y_train_pred_INGP[0].mean)
#print(y_test_pred_INGP[0].mean)
#print(y_train_pred_INGP[2].variance)
#print(y_test_pred_INGP[2].variance)
#print(y_train_pred_INGP[2].covariance_matrix)
#print(y_test_pred_INGP[2].covariance_matrix)

#y_train_pred_INGP = y_train_pred_INGP.cpu().detach().numpy()
#y_test_pred_INGP = y_test_pred_INGP.cpu().detach().numpy()

### 2.2.3 INGP prediction result analysis & model performance evaluation

In [29]:
# Calculate R2 value
from sklearn.metrics import r2_score
R2_train = r2_score(np_y_train[:,cond:(cond+d)], y_train_pred_INGP_mean)
R2_test = r2_score(np_y_test[:,cond:(cond+d)], y_test_pred_INGP_mean)
print("R2_train: {R2}".format(R2 = R2_train))
print("R2_test: {R2}".format(R2 = R2_test))

# Calculate MAPE value
from sklearn.metrics import mean_absolute_percentage_error
MAPE_train = mean_absolute_percentage_error(np_y_train[:,cond:(cond+d)], y_train_pred_INGP_mean)
MAPE_test = mean_absolute_percentage_error(np_y_test[:,cond:(cond+d)], y_test_pred_INGP_mean)
print("MAPE_train: {MAPE}".format(MAPE = MAPE_train))
print("MAPE_test: {MAPE}".format(MAPE = MAPE_test))

R2_train: -0.02116513046154589
R2_test: -0.027252932605054347
MAPE_train: 396127700113.412
MAPE_test: 360385345327.46783


## 2.2.4 INGP Model Save & Predict for Later Optimization

### Save Model

In [ ]:
ls_net = INGP.lengthscale_net.state_dict()
sc_net = INGP.scale_net.state_dict()

In [ ]:
torch.save({"ls_net": ls_net, "sc_net": sc_net}, "MLP.pt")

### Get Newest Inputs: Comany_id + 30-d Prices

In [41]:
raw = np.loadtxt(open("prices_final.csv","rb"), delimiter=",")

In [51]:
companies = np.unique(ts_data[:,0]).reshape(20,1)
z_final = raw[:,(raw.shape[1]-30):]
print(type(z_final))
z_final = np.concatenate((companies, z_final), axis=1)
print(type(z_final))
z_final = (z_final - np.mean(z, axis=0)) / np.std(z, axis=0)
z_final = torch.tensor(z_final)
print(z_final.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
torch.Size([20, 31])


In [52]:
x_final = x_test[:20, :]
print(x_final.shape)

torch.Size([20, 30])


In [58]:
Return_mean, Return_cov = INGP.predict(x_final, z_final, cond=cond, d=1)

In [60]:
#Return_mean = Return_mean*np.std(y, axis=0)[0] + np.mean(y, axis=0)[0]
np.savetxt("Return_mean.csv", Return_mean, delimiter=',')